<a href="https://colab.research.google.com/github/pds2021/a5-nicoelbert/blob/assignment_5/fwf_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -Uqq fastai  # upgrade fastai on colab
from fastai.vision.all import *
from fastai.vision.widgets import *
import os, shutil

     |████████████████████████████████| 194kB 7.7MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Uni/PDS/FWF

path = Path(r'/content/drive/MyDrive/Uni/PDS/FWF')
data_path = Path(r'/content/drive/MyDrive/Uni/PDS/FWF/data')
data_path.ls()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Uni/PDS/FWF


(#240) [Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AF'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AL'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/DZ'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AS'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AD'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AO'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AI'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AQ'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AG'),Path('/content/drive/MyDrive/Uni/PDS/FWF/data/AR')...]

In [29]:
#load Masterdata
df_countries = pd.read_csv('/content/drive/MyDrive/Uni/PDS/FWF/countries.CSV')


dict_countrynames = df_countries.set_index('Code').to_dict()['Country']
list_countrycodes= df_countries['Code'].tolist()


In [62]:
def getPreText(confidence):
  #return the right preffix text based on the confidence
  pretext = ""
  if confidence >= 0.9:
    pretext = "This is the flag of "
  elif confidence >= 0.8:
    pretext = "I am pretty sure, that this is the flag of "
  elif confidence >= 0.7:
    pretext = "This should be the flag of "
  elif confidence >= 0.6:
    pretext = "This might be the flag of "
  elif confidence < 0.4:
    pretext = "I have to guess - eventually the flag of "
  return pretext

def getAfterText(confidence):
  #return right suffix
  c_percentage = "{} %".format(round(confidence * 100,1))
  aftertext = ". - I can say this with a confidence of " + c_percentage
  return aftertext


def on_click_classify(change):
  #get image
  img = PILImage.create(btn_upload.data[-1])
  out_pl = widgets.Output()
  out_pl.clear_output()
  with out_pl:display(img.to_thumb(256,256))

  #make prediction and retrieve information
  country_id = learn_inf.predict(img)[0]
  country_tensor_index = learn_inf.predict(img)[1].item()
  confidence = learn_inf.predict(img)[2].numpy()[country_tensor_index]

  #read full country name 
  country_name = dict_countrynames[country_id]

  #get fitting strings
  if confidence > 0:
    pretext= getPreText(confidence)
    aftertext = getAfterText(confidence)
  else:
    pretext= "No prediction possible"
    aftertext = ""
  print(pretext + country_name + aftertext)


In [60]:
from fastai.vision.widgets import *
learn_inf = load_learner(path/'export.pkl')

btn_upload = widgets.FileUpload()
btn_upload



FileUpload(value={}, description='Upload')

In [63]:
btn_run = widgets.Button(description='Classify')
btn_run.on_click(on_click_classify)
btn_run

Button(description='Classify', style=ButtonStyle())

I have to guess - eventually the flag of Singapore. - I can say this with a confidence of 28.1 %


I have to guess - eventually the flag of Afghanistan. - I can say this with a confidence of 22.7 %
